<a href="https://colab.research.google.com/github/WalterMaranini/ESP32_EsempioAggiornamentoFirmwareOTA/blob/main/RichiamoLLAMAsuCOLAB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
%%capture
!pip install "unsloth[torch]" fastapi uvicorn[standard] pyngrok


In [3]:
from unsloth import FastLanguageModel
import torch

MODEL_PATH = "/content/drive/MyDrive/llama3-ft-lora"  # adatta al tuo path
MAX_SEQ_LENGTH = 4096
DTYPE = None
LOAD_IN_4BIT = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name     = MODEL_PATH,
    max_seq_length = MAX_SEQ_LENGTH,
    dtype          = DTYPE,
    load_in_4bit   = LOAD_IN_4BIT,
)

FastLanguageModel.for_inference(model)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.4: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Unsloth 2025.11.4 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
       

In [4]:
%%writefile server.py
import torch
from fastapi import FastAPI
from pydantic import BaseModel
from unsloth import FastLanguageModel
from transformers import TextStreamer

# ⚠️ Stesso path usato sopra
MODEL_PATH = "/content/drive/MyDrive/llama3-ft-lora"

MAX_SEQ_LENGTH = 4096
DTYPE = None
LOAD_IN_4BIT = True

app = FastAPI()

print("Carico modello...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name     = MODEL_PATH,
    max_seq_length = MAX_SEQ_LENGTH,
    dtype          = DTYPE,
    load_in_4bit   = LOAD_IN_4BIT,
)
FastLanguageModel.for_inference(model)
print("Modello caricato.")

class GenerateRequest(BaseModel):
    prompt: str
    max_new_tokens: int = 256
    temperature: float = 0.7

@app.post("/generate")
def generate(req: GenerateRequest):
    inputs = tokenizer(
        req.prompt,
        return_tensors="pt",
    ).to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=req.max_new_tokens,
            do_sample=True,
            temperature=req.temperature,
        )

    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return {"text": text}


Writing server.py


In [5]:
from pyngrok import ngrok

NGROK_AUTH_TOKEN = "2pZwMFu3mJckr6XujeNXrhafHiQ_7j6uphWPjaKMMgQj46Nha"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

public_url = ngrok.connect(11434, "http")
public_url


<NgrokTunnel: "https://cf01d50b10c7.ngrok-free.app" -> "http://localhost:11434">